In [ ]:
# ==============================================================================
# 文件名: 02_model_tuning_xgb.py
# 描述: 针对 CDFT + RDKT 组合特征的 QSAR 模型超参数调优 (XGBoost)
# 项目: QSAR_Project
# ==============================================================================

import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold
from sklearn.pipeline import Pipeline
from scipy.stats import loguniform, randint, uniform

def run_xgb_tuning():
    # ---------------------------------------------------------
    # 1. 路径设置与数据加载
    # ---------------------------------------------------------
    input_path = 'data/processed/train_data.csv'
    results_dir = 'results/models'
    os.makedirs(results_dir, exist_ok=True)

    if not os.path.exists(input_path):
        print(f"❌ 错误: 找不到训练数据文件 {input_path}")
        return

    df_train = pd.read_csv(input_path)

    # 分离特征与目标值
    # 假设 Target_Log1o2 是你的目标变量，SMILES 是标识符
    X_train = df_train.drop(columns=['Target_Log1o2', 'SMILES'])
    y_train = df_train['Target_Log1o2']

    print(f">>> 已加载训练数据: {X_train.shape[0]} 样本, {X_train.shape[1]} 特征")

    # ---------------------------------------------------------
    # 2. 构建 XGBoost Pipeline
    # ---------------------------------------------------------
    # 注意：XGBoost 对特征缩放不敏感，但为了 Pipeline 完整性保留结构
    xgb_pipeline = Pipeline([
        ('xgb', XGBRegressor(
            objective='reg:squarederror',
            tree_method='hist',  # 使用直方图优化算法，速度更快
            n_jobs=-1,
            random_state=42
        ))
    ])

    # ---------------------------------------------------------
    # 3. 定义超参数搜索空间 (RandomizedSearch)
    # ---------------------------------------------------------
    param_dist = {
        'xgb__max_depth': [3, 4, 5, 6],
        'xgb__learning_rate': loguniform(0.01, 0.2),
        'xgb__n_estimators': randint(100, 1000),
        'xgb__subsample': uniform(0.6, 0.4),          # 0.6 ~ 1.0
        'xgb__colsample_bytree': uniform(0.6, 0.4),   # 0.6 ~ 1.0
        'xgb__min_child_weight': randint(1, 8),
        'xgb__gamma': uniform(0, 0.5),                # 增加保守度
        'xgb__reg_lambda': loguniform(0.1, 10.0),     # L2 正则
        'xgb__reg_alpha': [0, 0.01, 0.1, 1.0],        # L1 正则
    }

    # ---------------------------------------------------------
    # 4. 执行交叉验证与随机搜索
    # ---------------------------------------------------------
    print('>>> [Step 2] 正在进行 XGBoost 超参数调优 (CDFT+RDKT)...')

    # 采用 5 折交叉验证，重复 5 次以确保评估的稳定性
    rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

    search = RandomizedSearchCV(
        estimator=xgb_pipeline,
        param_distributions=param_dist,
        n_iter=80,                # 搜索次数，可根据计算能力调整
        cv=rkf,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        random_state=42,
        verbose=1,
        return_train_score=True
    )

    search.fit(X_train, y_train)

    # ---------------------------------------------------------
    # 5. 结果分析与保存
    # ---------------------------------------------------------
    best_params = search.best_params_
    best_rmse = np.sqrt(-search.best_score_)

    print('\n' + '='*30)
    print(f"🏆 最佳参数: {best_params}")
    print(f"📊 最佳交叉验证 RMSE: {best_rmse:.4f}")
    print('='*30)

    # 保存最佳参数文件
    params_file = os.path.join(results_dir, 'best_params_xgb.json')
    with open(params_file, 'w') as f:
        json.dump(best_params, f, indent=4)

    # 保存调优全记录 (CSV) 方便后续分析过拟合情况
    cv_results = pd.DataFrame(search.cv_results_)
    cv_results.to_csv(os.path.join(results_dir, 'xgb_tuning_history.csv'), index=False)

    # ---------------------------------------------------------
    # 6. 特征重要性初步观察 (可选)
    # ---------------------------------------------------------
    best_model = search.best_estimator_.named_steps['xgb']
    importances = pd.Series(best_model.feature_importances_, index=X_train.columns)
    top_10 = importances.sort_values(ascending=False).head(10)

    print("\n💡 前 10 名重要特征:")
    print(top_10)

    print(f'\n✅ 调优完成！参数已保存至: {params_file}')
    print('🚀 请运行脚本 03 进行最终模型训练与外部测试集验证。')

if __name__ == "__main__":
    run_xgb_tuning()